# Model Setup

In [1]:
import lymph
from scipy.special import factorial
import numpy as np
import pandas as pd
import scipy as sp
import emcee

graph = {
    ('tumor', 'primary')  : ['I','II', 'III', 'IV','V', 'VII'],
    ('lnl'  , 'I') :        ['II'],
    ('lnl'  , 'II') :       ['III'], 
    ('lnl'  , 'III'):       ['IV'], 
    ('lnl'  , 'IV') :       ['V'],
    ('lnl'  , 'V') :        [],
    ('lnl'  , 'VII') :      [],
}

model = lymph.MidlineBilateral(graph = graph,use_mixing= True, trans_symmetric =True)
model.modalities = {'CT': [0.76, 0.81],
                    'MRI': [0.63, 0.81],
                    'PET': [0.86, 0.79],
                    'FNA': [0.98, 0.80],
                    'diagnostic_consensus': [0.86, 0.81],
                    'pathology': [1.0, 1.0],
                    'pCT': [0.86, 0.81],
                    'max_llh': [1.0, 1.0]
                    }


# Time prior with p(early) = 0.3
def binom_pmf(k: np.ndarray, n: int, p: float):
    """Binomial PMF"""
    if p > 1. or p < 0.:
        raise ValueError("Binomial prob must be btw. 0 and 1")
    q = (1. - p)
    binom_coeff = factorial(n) / (factorial(k) * factorial(n - k))
    return binom_coeff * p**k * q**(n - k)

def parametric_binom_pmf(n: int):
    """Return a parametric binomial PMF"""
    def inner(t, p):
        """Parametric binomial PMF"""
        return binom_pmf(t, n, p)
    return inner

max_t = 10
model.diag_time_dists["early"] = sp.stats.binom.pmf(np.arange(max_t+1), max_t, 0.3)
model.diag_time_dists["late"] = parametric_binom_pmf(max_t)
model.modalities = {'treatment_diagnose': [1,0.81]}
# model.patient_data = dataset_full

## Classical analysis example

In [2]:
midline_backend = emcee.backends.HDFBackend(filename = "../lynference/models/samples2.hdf5")
samples_midline_model = midline_backend.get_chain(flat = True)

In [3]:
from sparing_scripts import sample_from_flattened
samples_reduced = sample_from_flattened(samples_midline_model, num_samples = 203, spaced = True,step_size = 89)

In [4]:
model.modalities = {'treatment_diagnose': [1,0.81]}

In [5]:
from sparing_scripts import risk_sampled, levels_to_spare, ci_single

diagnose = {"treatment_diagnose": {'ipsi':{
        "I": 0,
        "II": 0,
        "III": 0,
        "IV": 0,
        "V": 0,
        "VII": 0
    },
    'contra':{
        "I": 0,
        "II": 0,
        "III": 0,
        "IV": 0,
        "V": 0,
        "VII": 0
    }}}

sampled_risks, mean_risk = risk_sampled(samples_reduced, model, 'early', given_diagnoses = diagnose, midline_extension = False)
spared_lnls, total_risk, ranked_combined, treated_lnls,treated_array, treated_ipsi, treated_contra, sampled_total_risk = levels_to_spare(threshold = 0.1, model = model, mean_risks = mean_risk, sampled_risks = sampled_risks, ci = True)


In [6]:
print("to spare: ", spared_lnls)
print("total risk: ", np.round(100*total_risk,2), "%")
print("confidence interval: ", np.round(100*ci_single(sampled_total_risk), 2), "%")

to spare:  [('contra I', 0.00014041503678369102), ('contra V', 0.00025355191154452126), ('contra III', 0.0007148542767304306), ('contra IV', 0.0007899036948606521), ('contra VII', 0.00235240305833678), ('ipsi IV', 0.004841190373359489), ('ipsi V', 0.006446829344689972), ('contra II', 0.008457191928417764), ('ipsi I', 0.008707130663085186), ('ipsi VII', 0.008858032230079348), ('ipsi III', 0.03658636444627596)]
total risk:  7.36 %
confidence interval:  [6.39 8.23] %


## Combination analysis
Here we can also specifically analyze all combinations we encounter in a dataset

In [7]:
#load data and modify it for analysis
dataset_USZ =  pd.read_csv("data/cleanedUSZ.csv", header=[0,1,2]) #import data

maxllh =  dataset_USZ['max_llh']
t_stage = dataset_USZ['info']
ipsi = maxllh.loc[:,'ipsi'].drop(['IIa','IIb','VIII','Ib','IX','VI','X','Ia'],axis = 1)[['I','II','III','IV','V','VII']]
contra = maxllh.loc[:,'contra'].drop(['IIa','IIb','VIII','Ib','IX','VI','X','Ia'],axis = 1)[['I','II','III','IV','V','VII']]
ipsi_header = header = pd.MultiIndex.from_product([ ['ipsi'], ['I','II','III','IV','V','VII']], names=['', ''])
contra_header = pd.MultiIndex.from_product([['contra'], ['I','II','III','IV','V','VII']], names=['', ''])
ipsi.columns = ipsi_header
contra.columns = contra_header

dataset_analyze = pd.concat([t_stage,ipsi,contra],axis = 1)

In [8]:
dataset_analyze

tumor                     ipsi                                   contra  \
    t_stage midline_extension      I    II    III     IV      V    VII      I   
0      late              True  False  True  False  False  False  False  False   
1     early             False  False  True  False  False  False  False  False   
2      late              True   True  True   True   True  False   True   True   
3      late              True  False  True   True   True  False   True  False   
4     early             False  False  True  False  False  False   True  False   
..      ...               ...    ...   ...    ...    ...    ...    ...    ...   
282    late              True   True  True   True  False  False  False  False   
283    late             False  False  True   True  False  False  False  False   
284    late              True  False  True   True  False   True  False  False   
285    late              True  False  True   True  False  False  False  False   
286   early             False  False  True   True  False  False  False  False   

                                        
        II    III     IV      V    VII  
0    False  False  False  False  False  
1     True  False  False  False  False  
2     True   True   True  False  False  
3    False  False  False  False  False  
4    False  False  False  False  False  
..     ...    ...    ...    ...    ...  
282  False  False  False  False  False  
283  False  False  False  False  False  
284   True   True  False  False   True  
285   True  False  False  False  False  
286  False  False  False  False  False  

[287 rows x 14 columns]

In [9]:
from collections import Counter
from collections import defaultdict
# Convert the DataFrame to a NumPy array for easier manipulation
data = np.array(dataset_analyze)

# we extract all observed diagnoses
entry_combinations_with_indexes = defaultdict(list)
for index, row in enumerate(data):
    combination = tuple(row)
    entry_combinations_with_indexes[combination].append(index)
USZ_counts = []
USZ_combinations = []
USZ_indexes = []
for combination, indexes in entry_combinations_with_indexes.items():
    count = len(indexes)
    USZ_indexes.append(indexes)
    USZ_counts.append(count)
    USZ_combinations.append(combination)

In [34]:
from sparing_scripts import analysis_treated_lnls_combinations
usz_treated_lnls_no_risk, usz_treated_lnls_all, usz_treatment_array, usz_top3_spared, usz_total_risks, usz_treated_ipsi, usz_treated_contra, usz_sampled_risks_array, lnls_ranked, cis = analysis_treated_lnls_combinations(USZ_combinations, samples_reduced, model)

here we extract some extra information to produce a nice table

In [12]:
lnls = ['I','II', 'III', 'IV','V', 'VII']
t_stage = []
midline_extension = []
invovlvement_ipsi_USZ = []
invovlvement_contra_USZ = []
for diagnose_type in USZ_combinations:
    involved_ipsi = []
    involved_contra = []
    t_stage.append(diagnose_type[0])
    midline_extension.append(diagnose_type[1])
    for lnl_looper, involved_level in enumerate(lnls):
        if diagnose_type[lnl_looper +2] == True:
            involved_ipsi.append(involved_level) 
        if diagnose_type[lnl_looper +8] == True:
            involved_contra.append(involved_level)
    invovlvement_ipsi_USZ.append(involved_ipsi)
    invovlvement_contra_USZ.append(involved_contra)

In [16]:
data_export_usz = pd.DataFrame({'Percentage of patients': np.array(USZ_counts)/287,
                                'T-stage': t_stage,
                                'Midline Extension': midline_extension,
                                'Involvement Ipsi' : invovlvement_ipsi_USZ,
                                'Involvement Contra': invovlvement_contra_USZ,
                                'Treated Ipsi':  usz_treated_ipsi,
                                'Treated Contra': usz_treated_contra,
                                'risk': usz_total_risks,
                                'lower bound': cis[0],
                                'upper bound': cis[1],
                                'top 3 spared lnls risk': usz_top3_spared,
                                'lnls ranked': lnls_ranked
})

In [ ]:
data_export_usz

,Percentage of patients,T-stage,Midline Extension,Involvement Ipsi,Involvement Contra,Treated Ipsi,Treated Contra,risk,lower bound,upper bound,top 3 spared lnls risk
0,0.048780,late,True,[II],[],"[III, II]",[II],0.077791,0.067740,0.089008,"[(ipsi I, 0.020928523675987565), (ipsi VII, 0...."
1,0.010453,early,False,[II],[II],"[III, II]","[III, II]",0.061750,0.053847,0.071133,"[(ipsi I, 0.020669208427492723), (ipsi VII, 0...."
2,0.003484,late,True,"[I, II, III, IV, VII]","[I, II, III, IV]","[V, I, II, III, IV, VII]","[I, II, III, IV]",0.053768,0.032256,0.078497,"[(contra V, 0.042696622289579954), (contra VII..."
3,0.003484,late,True,"[II, III, IV, VII]",[],"[V, VII, II, III, IV]",[II],0.064973,0.054992,0.074914,"[(ipsi I, 0.029491262610923226), (contra III, ..."
4,0.010453,early,False,"[II, VII]",[],"[III, VII, II]",[],0.059468,0.051073,0.068677,"[(ipsi I, 0.021163655070474168), (contra II, 0..."
...,...,...,...,...,...,...,...,...,...,...,...
72,0.003484,early,False,"[II, IV]",[],"[V, III, II, IV]",[],0.057391,0.048247,0.066374,"[(ipsi I, 0.022340680849047168), (ipsi VII, 0...."
73,0.006969,late,False,"[II, III, V]",[],"[IV, II, III, V]",[],0.071844,0.061416,0.081779,"[(ipsi I, 0.028089405023911166), (ipsi VII, 0...."
74,0.003484,late,True,"[II, III]","[II, III, IV]","[I, IV, II, III]","[V, II, III, IV]",0.061302,0.051388,0.071071,"[(ipsi VII, 0.02290127826783818), (ipsi V, 0.0..."
75,0.003484,late,False,"[II, V]",[],"[IV, III, II, V]",[],0.063624,0.054131,0.072994,"[(ipsi I, 0.024800916851586873), (ipsi VII, 0...."


## risk tables
here is an example for calculating the risk of each combination to show up


In [ ]:
midline_backend = emcee.backends.HDFBackend(filename = "../lynference/models/samples2.hdf5")
samples_midline_model = midline_backend.get_chain(flat = True)

In [ ]:
from sparing_scripts import sample_from_flattened
samples_reduced = sample_from_flattened(samples_midline_model, num_samples = 203, spaced = True,step_size = 89)

In [ ]:
sampled_risks_early_no_ext, mean_risk_early_no_ext = risk_sampled(samples_reduced, model, 'early', midline_extension = False, given_diagnoses = None) 
sampled_risks_early_ext, mean_risk_early_ext = risk_sampled(samples_reduced, model, 'early', midline_extension = True, given_diagnoses = None)
sampled_risks_late_no_ext, mean_risk_late_no_ext = risk_sampled(samples_reduced, model, 'late', midline_extension = False, given_diagnoses = None)
sampled_risks_late_ext, mean_risk_late_ext = risk_sampled(samples_reduced, model, 'late', midline_extension = True, given_diagnoses = None)

In [ ]:
state_list = model.noext.ipsi.state_list
state_list_names = []
for index, state in enumerate(state_list):
    state_list_names.append(str(state))
df_early_no_ext = pd.DataFrame(mean_risk_early_no_ext, columns=state_list_names, index=state_list_names)
df_early_ext = pd.DataFrame(mean_risk_early_ext, columns=state_list_names, index=state_list_names)
df_late_no_ext = pd.DataFrame(mean_risk_late_no_ext, columns=state_list_names, index=state_list_names)
df_late_ext = pd.DataFrame(mean_risk_late_ext, columns=state_list_names, index=state_list_names)

# df_early_no_ext.to_excel('early_no_ext.xlsx')
# df_early_ext.to_excel('early_ext.xlsx')
# df_late_no_ext.to_excel('late_no_ext.xlsx')
# df_late_ext.to_excel('late_ext.xlsx')

## Full treatment table

In [185]:
from sparing_scripts import sample_from_flattened
samples_reduced = sample_from_flattened(samples_midline_model, num_samples = 100, spaced = False)
#note to exactly reproduce the trial results, the number of samples should be 203 and spaced = True, and step_size = 89 but for the sake of speed, we use 100 samples here

In [145]:
from itertools import product

# Generate all 2**14 combinations of 14 booleans
combinations = list(product([False, True], repeat=14))
combinations = [('early' if comb[0] == False else 'late',) + comb[1:] for comb in combinations]
print(combinations[0])
print(combinations[1])

('early', False, False, False, False, False, False, False, False, False, False, False, False, False)
('early', False, False, False, False, False, False, False, False, False, False, False, False, True)


In [78]:
import multiprocessing as mp

# Function to process a chunk of combinations
def process_combinations(chunk):
    return analysis_treated_lnls_combinations(chunk, samples_reduced, model)

# Divide the combinations into chunks
num_cores = mp.cpu_count() - 1
chunk_size = len(combinations) // num_cores
chunks = [combinations[i:i + chunk_size] for i in range(0, len(combinations), chunk_size)]

# Use multiprocessing to process the chunks
with mp.Pool(num_cores) as pool:
    results = pool.map(process_combinations, chunks)

# Combine the results from all chunks
treated_lnls_no_risk, treated_lnls_all, treatment_array, top3_spared, total_risks, treated_ipsi, treated_contra, sampled_risks_array, lnls_ranked, cis = zip(*results)

# Flatten the results
treated_lnls_no_risk = [item for sublist in treated_lnls_no_risk for item in sublist]
treated_lnls_all = [item for sublist in treated_lnls_all for item in sublist]
treatment_array = np.vstack(treatment_array)
top3_spared = [item for sublist in top3_spared for item in sublist]
total_risks = np.concatenate(total_risks)
treated_ipsi = [item for sublist in treated_ipsi for item in sublist]
treated_contra = [item for sublist in treated_contra for item in sublist]
sampled_risks_array = np.vstack(sampled_risks_array)
lnls_ranked = [item for sublist in lnls_ranked for item in sublist]
cis_lower = []
cis_upper = []
for item in cis:
    cis_lower.append(item[0])
    cis_upper.append(item[1])
flat_lower = [item for sublist in cis_lower for item in sublist]
flat_upper = [item for sublist in cis_upper for item in sublist]

compute the prevalence for each diagnosis


In [83]:
sampled_risks_early_no_ext, mean_risk_early_no_ext = risk_sampled(samples_reduced, model, 'early', midline_extension = False, given_diagnoses = None) 
sampled_risks_early_ext, mean_risk_early_ext = risk_sampled(samples_reduced, model, 'early', midline_extension = True, given_diagnoses = None)
sampled_risks_late_no_ext, mean_risk_late_no_ext = risk_sampled(samples_reduced, model, 'late', midline_extension = False, given_diagnoses = None)
sampled_risks_late_ext, mean_risk_late_ext = risk_sampled(samples_reduced, model, 'late', midline_extension = True, given_diagnoses = None)

In [180]:
#generate state list
state_list = np.array(np.meshgrid(*[[0, 1]] * 14)).T.reshape(-1, 14)
state_list = state_list[np.lexsort(np.fliplr(state_list).T)]
# Reshape the risk arrays into 1x4096 arrays
mean_risk_early_noext_flat = mean_risk_early_no_ext.reshape(-1)
mean_risk_early_ext_flat = mean_risk_early_ext.reshape(-1)
mean_risk_late_noext_flat = mean_risk_late_no_ext.reshape(-1)
mean_risk_late_ext_flat = mean_risk_late_ext.reshape(-1)
#combine them
full_risks = np.hstack([mean_risk_early_noext_flat, mean_risk_early_ext_flat, mean_risk_late_noext_flat, mean_risk_late_ext_flat])/4


generate some columns

In [181]:
lnls = ['I','II', 'III', 'IV','V', 'VII']
t_stage = []
midline_extension = []
invovlvement_ipsi = []
invovlvement_contra = []
for diagnose_type in combinations:
    involved_ipsi = []
    involved_contra = []
    t_stage.append(diagnose_type[0])
    midline_extension.append(diagnose_type[1])
    for lnl_looper, involved_level in enumerate(lnls):
        if diagnose_type[lnl_looper +2] == True:
            involved_ipsi.append(involved_level) 
        if diagnose_type[lnl_looper +8] == True:
            involved_contra.append(involved_level)
    invovlvement_ipsi.append(involved_ipsi)
    invovlvement_contra.append(involved_contra)

Produce and export data frame

In [182]:
data_export = pd.DataFrame({'Percentage of patients': full_risks,
                                'T-stage': t_stage,
                                'Midline Extension': midline_extension,
                                'Involvement Ipsi' : invovlvement_ipsi,
                                'Involvement Contra': invovlvement_contra,
                                'Treated Ipsi':  treated_ipsi,
                                'Treated Contra': treated_contra,
                                'risk': total_risks,
                                'lower bound': flat_lower,
                                'upper bound': flat_upper,
                                'top 3 spared lnls risk': top3_spared,
                                'lnls ranked': lnls_ranked
})

In [183]:
data_export.to_csv('full_combination_table_100_samples.csv', index=False)